In [83]:
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'

In [84]:
temp = lambda col: col not in ['payer_code']
df = pd.read_csv('dataset/pre_processed.csv', na_values='?', usecols=temp)
y = df['diabetesMed']
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),[75-100),1.000000,1.000000,1.000000,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),[75-100),1.000000,1.000000,7.000000,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),[75-100),1.000000,1.000000,7.000000,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),[75-100),1.000000,1.000000,7.000000,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),[75-100),2.000000,1.000000,7.000000,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),[75-100),3.000000,1.000000,2.000000,3,...,No,Steady,No,No,No,No,No,No,Yes,>30
6,55842,84259809,Caucasian,Male,[60-70),[75-100),1.000000,1.000000,2.000000,4,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,63768,114882984,Caucasian,Male,[70-80),[75-100),2.000000,1.000000,7.000000,5,...,No,No,No,No,No,No,No,No,Yes,>30
8,12522,48330783,Caucasian,Female,[80-90),[75-100),3.000000,3.000000,4.000000,9,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,15738,63555939,Caucasian,Female,[90-100),[75-100),1.000000,1.000000,4.000000,7,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [85]:
def feature_engineering(data):
    df = data.copy()
    def delete_column(del_col):
        for col in del_col: df.drop(col, 1, inplace=True)
                    
    delete_column(['encounter_id', "patient_nbr", "medical_specialty"])
    
    df = df.loc[:, (df != df.iloc[0]).any()]
    excluded_col = ['gender', 'race', 'age']
    obj_col = [col for col in df.columns if df.dtypes[col] == 'O' and 
               len(df[col].unique()) > 2 and col not in excluded_col]
    delete_column(obj_col)
    
    df = pd.get_dummies(df)
    no_col = [col for col in df.columns if col.endswith("No")]
    delete_column(no_col)
    
    return df

In [86]:
df = feature_engineering(df)
df

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,age_[80-90),age_[90-100),acetohexamide_Steady,tolbutamide_Steady,troglitazone_Steady,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,change_Ch,diabetesMed_Yes
0,1.000000,1.000000,1.000000,1,41.000000,0.00000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.000000,1.000000,7.000000,3,59.000000,0.00000,18,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,1.000000,1.000000,7.000000,2,11.000000,1.00000,13,2,0,1,...,0,0,0,0,0,0,0,0,0,1
3,1.000000,1.000000,7.000000,2,44.000000,0.00000,16,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,2.000000,1.000000,7.000000,1,51.000000,1.00000,8,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5,3.000000,1.000000,2.000000,3,31.000000,0.00000,16,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,1.000000,1.000000,2.000000,4,70.000000,2.00000,21,0,0,0,...,0,0,0,0,0,0,0,0,1,1
7,2.000000,1.000000,7.000000,5,73.000000,3.00000,12,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,3.000000,3.000000,4.000000,9,68.000000,2.00000,28,0,0,0,...,1,0,0,0,0,0,0,0,1,1
9,1.000000,1.000000,4.000000,7,33.000000,0.00000,18,0,0,0,...,0,1,0,0,0,0,0,0,1,1
